In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn_pandas import DataFrameMapper

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, PReLU, LeakyReLU

import keras
import tensorflow as tf

from keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option("max.columns", None)

In [3]:
df = pd.read_csv("injuries_mixed_with_ranks.csv")
del df["Unnamed: 0"], df["visitor"], df["home"], df['ot'], df["id"], df["date"], df["attendance"]

In [4]:
df.head()

,start (et),pts_visitor,pts_home,visitor_rank_assists,visitor_rank_blocks,visitor_rank_deffeciency,visitor_rank_drebounds,visitor_rank_fouls,visitor_rank_ft_pct,visitor_rankoeffeciency,visitor_rank_opp_assists,visitor_rank_opp_blocks,visitor_rank_opp_drebounds,visitor_rank_opp_fouls,visitor_rank_opp_orebounds,visitor_rank_opp_ptsfastbreak,visitor_rank_opp_ptsfrom2,visitor_rank_opp_ptsfrom3,visitor_rank_opp_ptsinpaint,visitor_rank_opp_ptspergame,visitor_rank_opp_steals,visitor_rank_opp_threepointpct,visitor_rank_opp_turnovers,visitor_rank_opp_twopointpct,visitor_rank_orebounds,visitor_rank_ptsfrom2,visitor_rank_ptsfrom3,visitor_rank_ptsfromfastbreak,visitor_rank_ptsinpaint,visitor_rank_scoring,visitor_rank_steals,visitor_rank_threeptpct,visitor_rank_turnovers,visitor_rank_twoptpct,visitor_back_to_back,visitor_added_player,visitor_lost_player,home_rank_assists,home_rank_blocks,home_rank_deffeciency,home_rank_drebounds,home_rank_fouls,home_rank_ft_pct,home_rankoeffeciency,home_rank_opp_assists,home_rank_opp_blocks,home_rank_opp_drebounds,home_rank_opp_fouls,home_rank_opp_orebounds,home_rank_opp_ptsfastbreak,home_rank_opp_ptsfrom2,home_rank_opp_ptsfrom3,home_rank_opp_ptsinpaint,home_rank_opp_ptspergame,home_rank_opp_steals,home_rank_opp_threepointpct,home_rank_opp_turnovers,home_rank_opp_twopointpct,home_rank_orebounds,home_rank_ptsfrom2,home_rank_ptsfrom3,home_rank_ptsfromfastbreak,home_rank_ptsinpaint,home_rank_scoring,home_rank_steals,home_rank_threeptpct,home_rank_turnovers,home_rank_twoptpct,home_back_to_back,home_added_player,home_lost_player
0,7:00p,105,130,27,5,4,7,11,10,22,15,22,14,27,27,10,26,4,24,6,18,1,21,8,27,20,11,19,27,19,17,6,24,17,False,0.0,0.0,10,8,20,12,2,7,20,21,18,26,24,1,23,9,24,6,11,15,22,27,17,12,30,2,21,29,25,24,11,7,28,False,0.0,0.0
1,7:30p,94,82,25,21,22,29,20,13,27,10,3,13,7,24,8,25,7,26,20,28,26,5,18,25,28,15,7,30,26,4,8,27,25,False,1.0,2.0,24,29,5,10,3,30,17,5,19,19,5,2,9,17,6,20,4,12,8,15,22,1,24,14,25,17,22,28,18,9,30,False,0.0,0.0
2,7:30p,98,92,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,False,1.0,2.0,8,10,17,14,7,1,4,13,2,10,26,10,3,6,15,18,7,5,4,28,24,29,19,7,13,6,13,20,1,8,10,False,2.0,2.0
3,8:00p,103,94,13,1,13,17,22,24,25,9,29,20,25,30,12,24,13,16,24,16,5,4,16,5,8,17,9,3,9,10,22,25,24,False,0.0,0.0,18,27,29,27,14,19,18,27,13,25,6,15,25,22,30,23,30,22,27,24,28,19,11,13,16,11,10,23,14,16,19,False,0.0,1.0
4,8:30p,102,91,15,30,6,9,28,18,6,19,9,3,4,4,28,2,10,7,8,29,14,12,4,8,18,4,28,12,4,3,2,26,18,False,0.0,0.0,4,28,12,1,24,20,16,4,24,29,14,11,22,4,20,3,9,4,6,29,12,21,14,8,14,20,15,29,17,1,16,False,1.0,1.0


In [5]:
z = (df["pts_home"] > df["pts_visitor"])

In [6]:
df["home_team_won"] = 0

In [7]:
df["home_team_won"][z] = 1

In [8]:
df["home_team_won"].value_counts()

1    3528
0    2583
Name: home_team_won, dtype: int64

In [9]:
3528 / (3528+2583)

0.5773195876288659

In [10]:
df["start (et)"] = df["start (et)"].str.replace("p", "")

In [11]:
df["start (et)"] = df["start (et)"].astype(str)

In [12]:
target = "home_team_won"
X = df.drop(target, axis=1)
y = df[target]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [14]:
mapper = DataFrameMapper([
    ('start (et)', LabelBinarizer()), 
    (['visitor_rank_assists'], MinMaxScaler()), 
    (['visitor_rank_blocks'], MinMaxScaler()),
    (['visitor_rank_deffeciency'], MinMaxScaler()), 
    (['visitor_rank_drebounds'], MinMaxScaler()),
    (['visitor_rank_fouls'], MinMaxScaler()), 
    (['visitor_rank_ft_pct'], MinMaxScaler()), 
    (['visitor_rankoeffeciency'], MinMaxScaler()),
    (['visitor_rank_opp_assists'], MinMaxScaler()), 
    (['visitor_rank_opp_blocks'], MinMaxScaler()),
    (['visitor_rank_opp_drebounds'], MinMaxScaler()), 
    (['visitor_rank_opp_fouls'], MinMaxScaler()),
    (['visitor_rank_opp_orebounds'], MinMaxScaler()), 
    (['visitor_rank_opp_ptsfastbreak'], MinMaxScaler()),
    (['visitor_rank_opp_ptsfrom2'], MinMaxScaler()), 
    (['visitor_rank_opp_ptsfrom3'], MinMaxScaler()),
    (['visitor_rank_opp_ptsinpaint'], MinMaxScaler()), 
    (['visitor_rank_opp_ptspergame'], MinMaxScaler()),
    (['visitor_rank_opp_steals'], MinMaxScaler()), 
    (['visitor_rank_opp_threepointpct'], MinMaxScaler()),
    (['visitor_rank_opp_turnovers'], MinMaxScaler()), 
    (['visitor_rank_opp_twopointpct'], MinMaxScaler()),
    (['visitor_rank_orebounds'], MinMaxScaler()), 
    (['visitor_rank_ptsfrom2'], MinMaxScaler()),
    (['visitor_rank_ptsfrom3'], MinMaxScaler()), 
    (['visitor_rank_ptsfromfastbreak'], MinMaxScaler()),
    (['visitor_rank_ptsinpaint'], MinMaxScaler()), 
    (['visitor_rank_scoring'], MinMaxScaler()),
    (['visitor_rank_steals'], MinMaxScaler()), 
    (['visitor_rank_threeptpct'], MinMaxScaler()),
    (['visitor_rank_turnovers'], MinMaxScaler()), 
    (['visitor_rank_twoptpct'], MinMaxScaler()),
    ('visitor_back_to_back', LabelBinarizer()),
    (['visitor_added_player'], MinMaxScaler()),
    (['visitor_lost_player'], MinMaxScaler()),
    (['home_rank_assists'], MinMaxScaler()),
    (['home_rank_blocks'], MinMaxScaler()), 
    (['home_rank_deffeciency'], MinMaxScaler()), 
    (['home_rank_drebounds'], MinMaxScaler()),
    (['home_rank_fouls'], MinMaxScaler()), 
    (['home_rank_ft_pct'], MinMaxScaler()), 
    (['home_rankoeffeciency'], MinMaxScaler()),
    (['home_rank_opp_assists'], MinMaxScaler()), 
    (['home_rank_opp_blocks'], MinMaxScaler()),
    (['home_rank_opp_drebounds'], MinMaxScaler()), 
    (['home_rank_opp_fouls'], MinMaxScaler()),
    (['home_rank_opp_orebounds'], MinMaxScaler()), 
    (['home_rank_opp_ptsfastbreak'], MinMaxScaler()),
    (['home_rank_opp_ptsfrom2'], MinMaxScaler()), 
    (['home_rank_opp_ptsfrom3'], MinMaxScaler()),
    (['home_rank_opp_ptsinpaint'], MinMaxScaler()), 
    (['home_rank_opp_ptspergame'], MinMaxScaler()),
    (['home_rank_opp_steals'], MinMaxScaler()), 
    (['home_rank_opp_threepointpct'], MinMaxScaler()),
    (['home_rank_opp_turnovers'], MinMaxScaler()), 
    (['home_rank_opp_twopointpct'], MinMaxScaler()),
    (['home_rank_orebounds'], MinMaxScaler()), 
    (['home_rank_ptsfrom2'], MinMaxScaler()), 
    (['home_rank_ptsfrom3'], MinMaxScaler()),
    (['home_rank_ptsfromfastbreak'], MinMaxScaler()), 
    (['home_rank_ptsinpaint'], MinMaxScaler()),
    (['home_rank_scoring'], MinMaxScaler()), 
    (['home_rank_steals'], MinMaxScaler()), 
    (['home_rank_threeptpct'], MinMaxScaler()),
    (['home_rank_turnovers'], MinMaxScaler()), 
    (['home_rank_twoptpct'], MinMaxScaler()),
    ('home_back_to_back', LabelBinarizer()),
    (['home_added_player'], MinMaxScaler()),
    (['home_lost_player'], MinMaxScaler()),
],df_out=True)

In [15]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [16]:
model = CatBoostClassifier().fit(Z_train, y_train)
print("Cat Boost train score is " + str(model.score(Z_train, y_train)))
print("Cat Boost test score is " + str(model.score(Z_test, y_test)))

Learning rate set to 0.019735
0:	learn: 0.6907794	total: 61.8ms	remaining: 1m 1s
1:	learn: 0.6886599	total: 64.9ms	remaining: 32.4s
2:	learn: 0.6867414	total: 68ms	remaining: 22.6s
3:	learn: 0.6845499	total: 71.2ms	remaining: 17.7s
4:	learn: 0.6822073	total: 74.3ms	remaining: 14.8s
5:	learn: 0.6804296	total: 77.4ms	remaining: 12.8s
6:	learn: 0.6785178	total: 80.5ms	remaining: 11.4s
7:	learn: 0.6767918	total: 83.5ms	remaining: 10.4s
8:	learn: 0.6750105	total: 86.7ms	remaining: 9.54s
9:	learn: 0.6734689	total: 89.6ms	remaining: 8.87s
10:	learn: 0.6716700	total: 92.6ms	remaining: 8.33s
11:	learn: 0.6698380	total: 95.7ms	remaining: 7.88s
12:	learn: 0.6680159	total: 98.8ms	remaining: 7.5s
13:	learn: 0.6664323	total: 102ms	remaining: 7.17s
14:	learn: 0.6648137	total: 105ms	remaining: 6.88s
15:	learn: 0.6632450	total: 108ms	remaining: 6.63s
16:	learn: 0.6618100	total: 111ms	remaining: 6.41s
17:	learn: 0.6603141	total: 114ms	remaining: 6.22s
18:	learn: 0.6587644	total: 117ms	remaining: 6.04s
1

160:	learn: 0.5727035	total: 581ms	remaining: 3.02s
161:	learn: 0.5723186	total: 586ms	remaining: 3.03s
162:	learn: 0.5719561	total: 590ms	remaining: 3.03s
163:	learn: 0.5716536	total: 595ms	remaining: 3.03s
164:	learn: 0.5713422	total: 602ms	remaining: 3.05s
165:	learn: 0.5709711	total: 609ms	remaining: 3.06s
166:	learn: 0.5705727	total: 615ms	remaining: 3.07s
167:	learn: 0.5700820	total: 621ms	remaining: 3.07s
168:	learn: 0.5696963	total: 627ms	remaining: 3.08s
169:	learn: 0.5693676	total: 630ms	remaining: 3.08s
170:	learn: 0.5689890	total: 633ms	remaining: 3.07s
171:	learn: 0.5686270	total: 638ms	remaining: 3.07s
172:	learn: 0.5681904	total: 644ms	remaining: 3.08s
173:	learn: 0.5679128	total: 649ms	remaining: 3.08s
174:	learn: 0.5674924	total: 654ms	remaining: 3.08s
175:	learn: 0.5671998	total: 657ms	remaining: 3.08s
176:	learn: 0.5669088	total: 660ms	remaining: 3.07s
177:	learn: 0.5665905	total: 663ms	remaining: 3.06s
178:	learn: 0.5663298	total: 666ms	remaining: 3.06s
179:	learn: 

323:	learn: 0.5226932	total: 1.14s	remaining: 2.38s
324:	learn: 0.5224067	total: 1.15s	remaining: 2.38s
325:	learn: 0.5221428	total: 1.15s	remaining: 2.38s
326:	learn: 0.5218844	total: 1.16s	remaining: 2.38s
327:	learn: 0.5216365	total: 1.16s	remaining: 2.38s
328:	learn: 0.5214393	total: 1.16s	remaining: 2.38s
329:	learn: 0.5212201	total: 1.17s	remaining: 2.37s
330:	learn: 0.5209075	total: 1.17s	remaining: 2.37s
331:	learn: 0.5205493	total: 1.17s	remaining: 2.36s
332:	learn: 0.5202574	total: 1.18s	remaining: 2.36s
333:	learn: 0.5201068	total: 1.18s	remaining: 2.36s
334:	learn: 0.5198517	total: 1.19s	remaining: 2.35s
335:	learn: 0.5195429	total: 1.19s	remaining: 2.35s
336:	learn: 0.5192031	total: 1.19s	remaining: 2.34s
337:	learn: 0.5189391	total: 1.19s	remaining: 2.34s
338:	learn: 0.5187436	total: 1.2s	remaining: 2.33s
339:	learn: 0.5184887	total: 1.2s	remaining: 2.33s
340:	learn: 0.5182889	total: 1.2s	remaining: 2.33s
341:	learn: 0.5180852	total: 1.21s	remaining: 2.32s
342:	learn: 0.5

496:	learn: 0.4738613	total: 1.71s	remaining: 1.74s
497:	learn: 0.4735115	total: 1.72s	remaining: 1.73s
498:	learn: 0.4732397	total: 1.72s	remaining: 1.73s
499:	learn: 0.4729485	total: 1.73s	remaining: 1.73s
500:	learn: 0.4725741	total: 1.73s	remaining: 1.72s
501:	learn: 0.4722587	total: 1.73s	remaining: 1.72s
502:	learn: 0.4720372	total: 1.74s	remaining: 1.72s
503:	learn: 0.4716565	total: 1.74s	remaining: 1.72s
504:	learn: 0.4712868	total: 1.75s	remaining: 1.71s
505:	learn: 0.4709049	total: 1.75s	remaining: 1.71s
506:	learn: 0.4705614	total: 1.75s	remaining: 1.71s
507:	learn: 0.4703080	total: 1.76s	remaining: 1.7s
508:	learn: 0.4700379	total: 1.76s	remaining: 1.7s
509:	learn: 0.4697789	total: 1.76s	remaining: 1.7s
510:	learn: 0.4695485	total: 1.77s	remaining: 1.69s
511:	learn: 0.4691441	total: 1.77s	remaining: 1.69s
512:	learn: 0.4688289	total: 1.77s	remaining: 1.68s
513:	learn: 0.4684754	total: 1.78s	remaining: 1.68s
514:	learn: 0.4681013	total: 1.78s	remaining: 1.68s
515:	learn: 0.4

656:	learn: 0.4258001	total: 2.29s	remaining: 1.2s
657:	learn: 0.4254579	total: 2.29s	remaining: 1.19s
658:	learn: 0.4251122	total: 2.3s	remaining: 1.19s
659:	learn: 0.4248265	total: 2.3s	remaining: 1.19s
660:	learn: 0.4245449	total: 2.31s	remaining: 1.18s
661:	learn: 0.4242903	total: 2.32s	remaining: 1.18s
662:	learn: 0.4240404	total: 2.32s	remaining: 1.18s
663:	learn: 0.4236790	total: 2.32s	remaining: 1.18s
664:	learn: 0.4233876	total: 2.33s	remaining: 1.17s
665:	learn: 0.4231112	total: 2.33s	remaining: 1.17s
666:	learn: 0.4228155	total: 2.34s	remaining: 1.17s
667:	learn: 0.4225905	total: 2.34s	remaining: 1.16s
668:	learn: 0.4223309	total: 2.35s	remaining: 1.16s
669:	learn: 0.4220569	total: 2.35s	remaining: 1.16s
670:	learn: 0.4218459	total: 2.36s	remaining: 1.16s
671:	learn: 0.4215546	total: 2.36s	remaining: 1.15s
672:	learn: 0.4212878	total: 2.36s	remaining: 1.15s
673:	learn: 0.4209828	total: 2.37s	remaining: 1.14s
674:	learn: 0.4207298	total: 2.37s	remaining: 1.14s
675:	learn: 0.4

829:	learn: 0.3803287	total: 2.87s	remaining: 587ms
830:	learn: 0.3803028	total: 2.87s	remaining: 584ms
831:	learn: 0.3801691	total: 2.88s	remaining: 581ms
832:	learn: 0.3799688	total: 2.88s	remaining: 578ms
833:	learn: 0.3797242	total: 2.89s	remaining: 575ms
834:	learn: 0.3795194	total: 2.89s	remaining: 571ms
835:	learn: 0.3793660	total: 2.89s	remaining: 568ms
836:	learn: 0.3791705	total: 2.9s	remaining: 564ms
837:	learn: 0.3788485	total: 2.9s	remaining: 561ms
838:	learn: 0.3786167	total: 2.9s	remaining: 557ms
839:	learn: 0.3783578	total: 2.9s	remaining: 553ms
840:	learn: 0.3780859	total: 2.91s	remaining: 550ms
841:	learn: 0.3778364	total: 2.91s	remaining: 546ms
842:	learn: 0.3775442	total: 2.92s	remaining: 543ms
843:	learn: 0.3773563	total: 2.92s	remaining: 539ms
844:	learn: 0.3771068	total: 2.92s	remaining: 536ms
845:	learn: 0.3768706	total: 2.92s	remaining: 532ms
846:	learn: 0.3766613	total: 2.93s	remaining: 529ms
847:	learn: 0.3763374	total: 2.93s	remaining: 525ms
848:	learn: 0.37

998:	learn: 0.3421556	total: 3.44s	remaining: 3.44ms
999:	learn: 0.3419065	total: 3.45s	remaining: 0us
Cat Boost train score is 0.9395592406720489
Cat Boost test score is 0.6544502617801047


In [17]:
df2 = pd.read_csv("daily_data/full_daily_ranks.csv")

In [25]:
df_away = df2[(df2["team"] == "LA Lakers")]
del df_away["Unnamed: 0"], df_away["team"]

i = "visitor"
df_away = df_away.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_away = df_away.reset_index()

df_home = df2[(df2["team"] == "LA Clippers")]
del df_home["Unnamed: 0"], df_home["team"]

i = "home"
df_home = df_home.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_home = df_home.reset_index()

df_predict = pd.concat([df_away, df_home], axis=1)
del df_predict["index"]

df_predict["start (et)"] = "10:00"
df_predict["visitor_added_player"] = 0
df_predict["visitor_lost_player"] = 1
df_predict["visitor_back_to_back"] = False
df_predict["home_added_player"] = 0
df_predict["home_lost_player"] = 1
df_predict["home_back_to_back"] = False

Z_new = mapper.transform(df_predict)
model.predict(Z_new)

array([1])